In [ ]:
# imports
from transformers import AutoModelForCausalLM, AutoTokenizer
from trl import DPOTrainer, DPOConfig, AutoModelForSeq2SeqLMWithValueHead, create_reference_model
import torch
from datasets import load_from_disk
from vc.encodec_model.nar_bart_model import NARBartForConditionalGeneration
from transformers import AutoTokenizer, BatchEncoding
from tqdm import tqdm

In [ ]:
ar_checkpoint = "lca0503/speech-chatgpt-base-ar-v2-epoch10-wotrans"
nar_checkpoint = "lca0503/speech-chatgpt-base-nar-v2-epoch4-wotrans"

device = "cuda" if torch.cuda.is_available() else "cpu"

# model = AutoModelForCausalLM.from_pretrained("gpt2")
# tokenizer = AutoTokenizer.from_pretrained("gpt2")
# tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForSeq2SeqLMWithValueHead.from_pretrained(ar_checkpoint, return_dict=True)
model_ref = create_reference_model(model)
nar_model = NARBartForConditionalGeneration.from_pretrained(nar_checkpoint)
ar_tokenizer = AutoTokenizer.from_pretrained(ar_checkpoint)
nar_tokenizer = AutoTokenizer.from_pretrained(nar_checkpoint)
ar_tokenizer.pad_token = ar_tokenizer.eos_token

In [ ]:
# from datasets import Dataset

# dpo_dataset_dict = {
#     "prompt": [
#         "hello",
#         "how are you",
#         "What is your name?",
#         "What is your name?",
#         "Which is the best programming language?",
#         "Which is the best programming language?",
#         "Which is the best programming language?",
#     ],
#     "chosen": [
#         "hi nice to meet you",
#         "I am fine",
#         "My name is Mary",
#         "My name is Mary",
#         "Python",
#         "Python",
#         "Java",
#     ],
#     "rejected": [
#         "leave me alone",
#         "I am not fine",
#         "Whats it to you?",
#         "I dont have a name",
#         "Javascript",
#         "C++",
#         "C++",
#     ],
# }

# dataset = Dataset.from_dict(dpo_dataset_dict)
# # print(dataset)

In [10]:
import json
from datasets import Dataset

# load the dpo_data.json file
with open("dpo_data.json") as f:
    dpo_data = json.load(f)
    
# load the dataset
dataset = Dataset.from_dict(dpo_data)
print(dataset)

## TODO: Split the dataset into training and validation sets
from datasets import DatasetDict

# split the dataset
dataset_dict = dataset.train_test_split(test_size=0.1)

# load the training and validation datasets
train_dataset = dataset_dict["train"]
val_dataset = dataset_dict["test"]
print("train_dataset", train_dataset)
print("val_dataset", val_dataset)

Dataset({
    features: ['prompt', 'chosen', 'rejected'],
    num_rows: 2
})
train_dataset Dataset({
    features: ['prompt', 'chosen', 'rejected'],
    num_rows: 1
})
val_dataset Dataset({
    features: ['prompt', 'chosen', 'rejected'],
    num_rows: 1
})


In [11]:
import os
import wandb
wandb.login()

os.environ["WANDB_NOTEBOOK_NAME"] = "trl"
wandb.init(project="trl")

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


In [12]:
training_args = DPOConfig(
    beta=0.1,
    output_dir="./results",
    generate_during_eval = True,
)

trainer = DPOTrainer(
    model=model,
    args=training_args,
    tokenizer=ar_tokenizer,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

# train
trainer.train()

/work/b0990106x/trl/trl/trainer/dpo_trainer.py:362: UserWarning: `max_length` is not set in the DPOConfig's init it will default to `512` by default, but you should do it yourself in the future.
  warnings.warn(
/work/b0990106x/trl/trl/trainer/dpo_trainer.py:375: UserWarning: `max_prompt_length` is not set in the DPOConfig's init it will default to `128` by default, but you should do it yourself in the future.
  warnings.warn(
/work/b0990106x/trl/trl/trainer/dpo_trainer.py:388: UserWarning: When using an encoder decoder architecture, you should set `max_target_length` in the DPOConfig's init it will default to `128` by default, but you should do it yourself in the future.
  warnings.warn(
/work/b0990106x/trl/trl/trainer/dpo_trainer.py:410: UserWarning: When using DPODataCollatorWithPadding, you should set `remove_unused_columns=False` in your TrainingArguments we have set it for you, but you should do it yourself in the future.
  warnings.warn(
Map: 100%|██████████| 1/1 [00:00<00:00, 9

Step,Training Loss


TrainOutput(global_step=3, training_loss=0.1769325534502665, metrics={'train_runtime': 0.297, 'train_samples_per_second': 10.101, 'train_steps_per_second': 10.101, 'total_flos': 0.0, 'train_loss': 0.1769325534502665, 'epoch': 3.0})